In [24]:
 #libraries used to extract, clean and manipulate the data
import pandas as pd 
import numpy as np
import string

#To plot the graphs
import matplotlib.pyplot as plt
plt.style.use('seaborn')

#library used to count the frequency of words
from sklearn.feature_extraction.text import CountVectorizer

#To create the sentiment analysis model, tokenization and lemmatization
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('vader_lexicon')
#nltk.download('punkt')

In [89]:
from string import punctuation

In [90]:
punctuation = set(punctuation)

In [26]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jackc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [28]:
df = pd.read_csv('songs_df.csv')

In [30]:
df = df.drop(columns= ['Unnamed: 0'])

In [31]:
df.head(10)

,Artist,Song,Genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,Lyrics
0,My Chemical Romance,Teenagers,"['emo', 'modern rock', 'pop punk', 'rock']",0.463,0.857,4,-3.063,1,0.0632,0.050600,0.000000,0.1840,0.8560,111.647,161920,4,82,teenagers lyrics they're gonna clean up your l...
1,My Chemical Romance,Welcome to the Black Parade,"['emo', 'modern rock', 'pop punk', 'rock']",0.217,0.905,2,-4.103,1,0.0752,0.000289,0.000110,0.2220,0.2360,96.950,311107,4,78,welcome to the black parade lyrics when i was ...
2,My Chemical Romance,I'm Not Okay (I Promise),"['emo', 'modern rock', 'pop punk', 'rock']",0.210,0.940,9,-3.427,1,0.1230,0.006020,0.000000,0.2690,0.2550,179.722,186480,4,75,im not okay i promise lyrics well if you wa...
3,My Chemical Romance,Helena,"['emo', 'modern rock', 'pop punk', 'rock']",0.356,0.960,4,-3.487,1,0.1040,0.014200,0.000000,0.2090,0.0857,125.921,204733,4,74,the entire mixtape lyricsring my bell anita w...
4,My Chemical Romance,Famous Last Words,"['emo', 'modern rock', 'pop punk', 'rock']",0.467,0.883,2,-3.679,0,0.1060,0.000321,0.000152,0.1250,0.3090,122.252,299227,4,72,famous last words lyrics now i know that i can...
5,My Chemical Romance,I Don't Love You,"['emo', 'modern rock', 'pop punk', 'rock']",0.289,0.796,0,-4.220,1,0.0494,0.008370,0.000026,0.2020,0.3300,169.835,238680,4,71,i dont love you lyrics well when you go don't ...
6,My Chemical Romance,Na Na Na (Na Na Na Na Na Na Na Na Na),"['emo', 'modern rock', 'pop punk', 'rock']",0.411,0.986,8,-2.522,0,0.0583,0.000264,0.001480,0.3710,0.6490,165.981,205960,4,70,na na na na na na na na na na na na lyrics dr ...
7,My Chemical Romance,Dead!,"['emo', 'modern rock', 'pop punk', 'rock']",0.481,0.973,7,-1.945,1,0.0542,0.003700,0.000877,0.3250,0.3620,99.331,195520,4,69,dead lyrics yeah and if your heart stops b...
8,My Chemical Romance,The Ghost of You,"['emo', 'modern rock', 'pop punk', 'rock']",0.202,0.886,11,-3.805,0,0.0816,0.028600,0.000000,0.6430,0.1920,145.781,194253,4,68,the ghost of you lyrics i never said i'd lie a...
9,My Chemical Romance,Mama,"['emo', 'modern rock', 'pop punk', 'rock']",0.236,0.799,4,-5.167,0,0.0939,0.013600,0.000000,0.0948,0.3030,208.951,279027,4,68,mama lyrics gerard way mama we all go to hell ...


In [35]:
def clean_lyrics(df,column):

    df = df
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
    df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
    df[column] = df[column].str.replace("\n"," ").str.replace(r"[^\w\d'\s]+","").str.replace("efil ym fo flah","")
    df[column] = df[column].str.strip()

    return df

In [32]:
def lyrics_to_words(document):
    """
    This function splits the text of lyrics to  single words, removing stopwords and doing the lemmatization to each word
    parameters:
    document: text to split to single words
    """
    stop_words = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stopwordremoval = " ".join([i for i in document.lower().split() if i not in stop_words])
    punctuationremoval = ''.join(ch for ch in stopwordremoval if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punctuationremoval.split())
    return normalized

In [33]:
def unique(list1):
   # intilize a null list
    unique_list = []
   # traverse for all elements
    for x in list1:
         # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
            
    return unique_list

In [73]:
df = clean_lyrics(df,'Lyrics')

TypeError: list indices must be integers or slices, not str

In [54]:
genres = df['Genres'].to_list()

In [61]:
genres_split = genres[0].split(',')
genres_split[1]

" 'modern rock'"

In [63]:
#Stores unique words of each lyrics song into a new column called words
#list used to store the words
words = []
#iterate trought each lyric and split unique words appending the result into the words list
df = df.reset_index(drop=True)
for word in df['Lyrics'].tolist():
    
    words.append(unique(lyrics_to_words(word).split()))
#create the new column with the information of words lists
df['Words'] = words
df.head()

,Artist,Song,Genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,Lyrics,Words
0,My Chemical Romance,Teenagers,"['emo', 'modern rock', 'pop punk', 'rock']",0.463,0.857,4,-3.063,1,0.0632,0.050600,0.000000,0.184,0.8560,111.647,161920,4,82,teenagers lyrics they're gonna clean up your l...,"[teenager, lyric, theyre, gonna, clean, look, ..."
1,My Chemical Romance,Welcome to the Black Parade,"['emo', 'modern rock', 'pop punk', 'rock']",0.217,0.905,2,-4.103,1,0.0752,0.000289,0.000110,0.222,0.2360,96.950,311107,4,78,welcome to the black parade lyrics when i was ...,"[welcome, black, parade, lyric, young, boy, fa..."
2,My Chemical Romance,I'm Not Okay (I Promise),"['emo', 'modern rock', 'pop punk', 'rock']",0.210,0.940,9,-3.427,1,0.1230,0.006020,0.000000,0.269,0.2550,179.722,186480,4,75,im not okay i promise lyrics well if you wa...,"[im, okay, promise, lyric, well, wanted, hones..."
3,My Chemical Romance,Helena,"['emo', 'modern rock', 'pop punk', 'rock']",0.356,0.960,4,-3.487,1,0.1040,0.014200,0.000000,0.209,0.0857,125.921,204733,4,74,the entire mixtape lyricsring my bell anita w...,"[entire, mixtape, lyricsring, bell, anita, war..."
4,My Chemical Romance,Famous Last Words,"['emo', 'modern rock', 'pop punk', 'rock']",0.467,0.883,2,-3.679,0,0.1060,0.000321,0.000152,0.125,0.3090,122.252,299227,4,72,famous last words lyrics now i know that i can...,"[famous, last, word, lyric, know, cant, make, ..."


In [108]:
new_genre_list = []

for genre in df['Genres']:
    genre_split = genre.split(',')
    print(genre_split[0])
    new_genre_list.append(genre_split[0])

['emo'
['emo'
['emo'
['emo'
['emo'
['emo'
['emo'
['emo'
['emo'
['emo'
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['new jersey indie']
['modern rock'
['modern rock'
['pop']
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['emo'
['emo'
['emo'
['emo'
['dance pop'
['emo'
['emo'
['emo'
['emo'
['emo rap'
['ohio hip hop'
['post-hardcore'
['post-hardcore'
['post-hardcore'
['post-hardcore'
['post-hardcore'
['post-hardcore'
['post-hardcore'
['emo'
['post-hardcore'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['modern rock'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['bow pop'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative m

['bedroom pop'
['bedroom pop'
['bedroom pop'
[]
[]
[]
[]
[]
[]
[]
[]
[]
['bedroom pop']
['bedroom pop']
['bedroom pop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['japanese electropop']
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['art pop'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['alternative metal'
['ambient folk'
['ambient folk'
['ambient folk'
['ambient folk'
['ambient folk'
['ambient folk'
['ambient folk'
['ambient folk'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['mathcore'
['ninten

In [110]:
clean_genre_list = []

for genre in new_genre_list:
    genre_punc = "".join([ch for ch in genre if ch not in punctuation])
    clean_genre_list.append(genre_punc)

In [111]:
clean_genre_list

['emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'new jersey indie',
 'modern rock',
 'modern rock',
 'pop',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'emo',
 'emo',
 'emo',
 'emo',
 'dance pop',
 'emo',
 'emo',
 'emo',
 'emo',
 'emo rap',
 'ohio hip hop',
 'posthardcore',
 'posthardcore',
 'posthardcore',
 'posthardcore',
 'posthardcore',
 'posthardcore',
 'posthardcore',
 'emo',
 'posthardcore',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'modern rock',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'bow pop',
 'alternative metal',
 'alternative metal',
 'al

In [112]:
df['Top Genre'] = clean_genre_list

In [113]:
df

,Artist,Song,Genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,Lyrics,Words,Top Genre
0,My Chemical Romance,Teenagers,"['emo', 'modern rock', 'pop punk', 'rock']",0.463,0.857,4,-3.063,1,0.0632,0.050600,0.000000,0.1840,0.8560,111.647,161920,4,82,teenagers lyrics they're gonna clean up your l...,"[teenager, lyric, theyre, gonna, clean, look, ...",emo
1,My Chemical Romance,Welcome to the Black Parade,"['emo', 'modern rock', 'pop punk', 'rock']",0.217,0.905,2,-4.103,1,0.0752,0.000289,0.000110,0.2220,0.2360,96.950,311107,4,78,welcome to the black parade lyrics when i was ...,"[welcome, black, parade, lyric, young, boy, fa...",emo
2,My Chemical Romance,I'm Not Okay (I Promise),"['emo', 'modern rock', 'pop punk', 'rock']",0.210,0.940,9,-3.427,1,0.1230,0.006020,0.000000,0.2690,0.2550,179.722,186480,4,75,im not okay i promise lyrics well if you wa...,"[im, okay, promise, lyric, well, wanted, hones...",emo
3,My Chemical Romance,Helena,"['emo', 'modern rock', 'pop punk', 'rock']",0.356,0.960,4,-3.487,1,0.1040,0.014200,0.000000,0.2090,0.0857,125.921,204733,4,74,the entire mixtape lyricsring my bell anita w...,"[entire, mixtape, lyricsring, bell, anita, war...",emo
4,My Chemical Romance,Famous Last Words,"['emo', 'modern rock', 'pop punk', 'rock']",0.467,0.883,2,-3.679,0,0.1060,0.000321,0.000152,0.1250,0.3090,122.252,299227,4,72,famous last words lyrics now i know that i can...,"[famous, last, word, lyric, know, cant, make, ...",emo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5113,The Safety Fire,Yellowism,['djent'],0.460,0.973,0,-5.993,1,0.0787,0.000127,0.010700,0.4240,0.2620,145.042,323387,4,18,yellowism lyricsput the man on the stand a se...,"[yellowism, lyricsput, man, stand, sentence, e...",djent
5114,The Safety Fire,Old Souls,['djent'],0.511,0.953,4,-6.050,0,0.0995,0.001490,0.000190,0.2120,0.2480,150.099,445947,4,18,old souls lyricsbare feet on a stool neck atta...,"[old, soul, lyricsbare, foot, stool, neck, att...",djent
5115,The Safety Fire,"I Am Time, The Destroyer",['djent'],0.402,0.424,0,-9.200,1,0.0445,0.073900,0.001430,0.0999,0.1410,129.986,209293,3,17,i am time the destroyer lyricswe live in but t...,"[time, destroyer, lyricswe, live, minute, end,...",djent
5116,The Safety Fire,Glass Crush,['djent'],0.259,0.954,9,-5.263,0,0.0675,0.000097,0.167000,0.1020,0.0713,101.627,308707,4,17,glass crush lyricsthey say torture him to ensu...,"[glass, crush, lyricsthey, say, torture, ensur...",djent


In [119]:
df_emo = df[df['Top Genre'] == 'emo']

In [120]:
df_emo

,Artist,Song,Genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,Lyrics,Words,Top Genre
0,My Chemical Romance,Teenagers,"['emo', 'modern rock', 'pop punk', 'rock']",0.463,0.857,4,-3.063,1,0.0632,0.050600,0.000000,0.1840,0.8560,111.647,161920,4,82,teenagers lyrics they're gonna clean up your l...,"[teenager, lyric, theyre, gonna, clean, look, ...",emo
1,My Chemical Romance,Welcome to the Black Parade,"['emo', 'modern rock', 'pop punk', 'rock']",0.217,0.905,2,-4.103,1,0.0752,0.000289,0.000110,0.2220,0.2360,96.950,311107,4,78,welcome to the black parade lyrics when i was ...,"[welcome, black, parade, lyric, young, boy, fa...",emo
2,My Chemical Romance,I'm Not Okay (I Promise),"['emo', 'modern rock', 'pop punk', 'rock']",0.210,0.940,9,-3.427,1,0.1230,0.006020,0.000000,0.2690,0.2550,179.722,186480,4,75,im not okay i promise lyrics well if you wa...,"[im, okay, promise, lyric, well, wanted, hones...",emo
3,My Chemical Romance,Helena,"['emo', 'modern rock', 'pop punk', 'rock']",0.356,0.960,4,-3.487,1,0.1040,0.014200,0.000000,0.2090,0.0857,125.921,204733,4,74,the entire mixtape lyricsring my bell anita w...,"[entire, mixtape, lyricsring, bell, anita, war...",emo
4,My Chemical Romance,Famous Last Words,"['emo', 'modern rock', 'pop punk', 'rock']",0.467,0.883,2,-3.679,0,0.1060,0.000321,0.000152,0.1250,0.3090,122.252,299227,4,72,famous last words lyrics now i know that i can...,"[famous, last, word, lyric, know, cant, make, ...",emo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,Reggie And The Full Effect,From Me 2 U,"['emo', 'pop punk', 'screamo']",0.425,0.985,9,-4.140,1,0.0883,0.000039,0.000000,0.2440,0.3950,109.263,85547,4,26,from me u lyricstodays your final day when i ...,"[u, lyricstodays, final, day, thought, would, ...",emo
4997,Reggie And The Full Effect,Thanx for Stayin,"['emo', 'pop punk', 'screamo']",0.496,0.991,2,-3.521,1,0.3470,0.019500,0.000000,0.4190,0.0928,150.066,190649,4,24,thanx for stayin lyricsmy friends could say th...,"[thanx, stayin, lyricsmy, friend, could, say, ...",emo
4998,Reggie And The Full Effect,What The Hell Is Contempt,"['emo', 'pop punk', 'screamo']",0.536,0.933,11,-1.951,1,0.0580,0.000035,0.005450,0.2640,0.3270,102.501,193253,4,24,what the hell is contempt lyricsso far so good...,"[hell, contempt, lyricsso, far, good, ohh, wel...",emo
4999,Reggie And The Full Effect,Happy V-Day,"['emo', 'pop punk', 'screamo']",0.185,0.973,2,-4.521,1,0.2020,0.005650,0.000000,0.1450,0.4500,174.094,237453,4,22,happy vday lyricsthe hrc with the rge sometim...,"[happy, vday, lyricsthe, hrc, rge, sometimes, ...",emo


In [126]:
#Create a new dataframe of all the  words used in lyrics and its decades
#list used to store the information
set_words = []
#Iterate trought each word and decade and stores them into the new lists
for i in df_emo.index:
    for word in df_emo['Words'].iloc[i]:
        set_words.append(word)
        #set_genres.append(df_emo['Top Genre'].iloc[i])
#create the new data frame  with the information of words and decade lists
words_df = pd.DataFrame({'words':set_words})

IndexError: single positional indexer is out-of-bounds

In [118]:
#Defined  your own Stopwords in case the clean data function does not remove all of them
stop_words = ['verse','im','get','1000','58','60','80','youre','youve',
              'guitar','solo','instrumental','intro','pre',"3", 'lyric']
# count the frequency of each word that aren't on the stop_words lists
cv = CountVectorizer(stop_words=stop_words)
#Create a dataframe called data_cv to store the the number of times the word was used in  a lyric based their decades
text_cv = cv.fit_transform(words_df['words'].iloc[:])
data_cv = pd.DataFrame(text_cv.toarray(),columns=cv.get_feature_names_out())
data_cv['genre'] = words_df['Top Genre']
#created a dataframe that Sums the ocurrence frequency of each word and group the result by decade
vect_words = data_cv.groupby('genre').sum().T
vect_words = vect_words.reset_index(level=0).rename(columns ={'index':'words'})
vect_words = vect_words.rename_axis(columns='')
#Save the data into a csv file
#vect_words.to_csv('words.csv',index=False)
#change the order of columns to order from the oldest to actual decade
vect_words

MemoryError: Unable to allocate 799. GiB for an array with shape (931003, 115231) and data type int64